In [ ]:
import pandas as pd
import numpy as np
import regex as re
from nltk.corpus import stopwords
import nltk
import gensim
from gensim.utils import simple_preprocess

In [ ]:
df = pd.read_csv('C:/Users/YANG/Desktop/projects/USvideos.csv')

In [ ]:
df.shape

(40949, 16)

In [ ]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6S...,17.14.11,WE WAN...,CaseyN...,22,2017-1...,SHANte...,748374,57527,2966,15954,https:...,False,False,False,SHANTE...
1,1ZAPwf...,17.14.11,The Tr...,LastWe...,24,2017-1...,last w...,2418783,97185,6146,12703,https:...,False,False,False,One ye...
2,5qpjK5...,17.14.11,Racist...,Rudy M...,23,2017-1...,racist...,3191434,146033,5339,8181,https:...,False,False,False,WATCH ...
3,puqaWr...,17.14.11,Nickel...,Good M...,24,2017-1...,rhett ...,343168,10172,666,2146,https:...,False,False,False,Today ...
4,d380me...,17.14.11,I Dare...,nigahiga,24,2017-1...,"ryan|""...",2095731,132235,1989,17518,https:...,False,False,False,I know...


In [ ]:
import json
with open('C:/Users/YANG/Desktop/projects/US_category_id.json','r') as f:
    category_data_us= json.loads(f.read())
US_cat = pd.json_normalize(category_data_us,record_path='items')

In [ ]:
US_cat.head()

,kind,etag,id,snippet.channelId,snippet.title,snippet.assignable
0,youtub...,"""m2ysk...",1,UCBR8-...,Film &...,True
1,youtub...,"""m2ysk...",2,UCBR8-...,Autos ...,True
2,youtub...,"""m2ysk...",10,UCBR8-...,Music,True
3,youtub...,"""m2ysk...",15,UCBR8-...,Pets &...,True
4,youtub...,"""m2ysk...",17,UCBR8-...,Sports,True


In [ ]:
US_cat['id']= US_cat['id'].astype(int)

In [ ]:
df= df.merge(US_cat,how ='left',left_on= 'category_id',\
                                 right_on='id').rename(columns= {'snippet.title':'category_name'})

In [ ]:
df.thumbnail_link

0        https:...
1        https:...
2        https:...
3        https:...
4        https:...
           ...    
40944    https:...
40945    https:...
40946    https:...
40947    https:...
40948    https:...
Name: thumbnail_link, Length: 40949, dtype: object

In [ ]:
df.drop(columns=['thumbnail_link','kind','etag','id','snippet.assignable','snippet.channelId'], axis='columns',inplace=True)

In [ ]:
df.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
0,2kyS6S...,17.14.11,WE WAN...,CaseyN...,22,2017-1...,SHANte...,748374,57527,2966,15954,False,False,False,SHANTE...,People...


In [ ]:
df.drop(columns=['comments_disabled','ratings_disabled'],axis=1,inplace=True)

In [ ]:
df.description.fillna('',inplace=True)

In [ ]:
def isEnglish(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

In [ ]:
def removeNonEnglishWords(text):
    filteredText = []
    for word in text.split():
        if isEnglish(word):
            filteredText.append(word)
    
    return " ".join(filteredText)

In [ ]:
df['des']= df.description.apply(removeNonEnglishWords)

In [ ]:
df.drop(columns='description',axis=1,inplace=True)

In [ ]:
df['c_tags']= df.tags.apply(removeNonEnglishWords)
df.drop(columns='tags',axis=1,inplace=True)

In [ ]:
df['c_title']= df.title.apply(removeNonEnglishWords)
df.drop(columns='title',axis=1,inplace=True)

In [ ]:
df['channeltitle']= df.channel_title.apply(removeNonEnglishWords)
df.drop(columns='channel_title',axis=1,inplace=True)

In [ ]:
pd.set_option('display.max_colwidth', None)
df.des.loc[10]

"New England Patriots returner Dion Lewis blasts off for an amazing kickoff return touchdown against the Denver Broncos in Week 10 of the 2017 NFL Season.\\n\\nWatch full games with NFL Game Pass: https://www.nfl.com/gamepass?campaign=sp-nf-gd-ot-yt-3000342\\n\\nSign up for Fantasy Football! http://www.nfl.com/fantasyfootball\\n\\nSubscribe to NFL: http://j.mp/1L0bVBu\\n\\nThe NFL YouTube channel is your home for immediate in-game highlights from your favorite teams and players, full NFL games, behind the scenes access and more!\\n\\nCheck out our other channels:\\nNFL Network http://www.youtube.com/nflnetwork\\nNFL Films http://www.youtube.com/nflfilms\\n\\nFor all things NFL, visit the league's official website at http://www.nfl.com/\\n\\nWatch NFL Now: https://www.nfl.com/now\\nListen to NFL podcasts: http://www.nfl.com/podcasts\\nWatch the NFL network: http://nflnonline.nfl.com/\\nDownload the NFL mobile app: https://www.nfl.com/apps\\n2017 NFL Schedule: http://www.nfl.com/schedule

In [ ]:
df.des= df.des.str.replace('http\S+|www.\S+','',regex=True).str.replace('\r+','',regex= True).str.lower()#remove links
df.des = df.des.str.replace(r'[^\w\s]+', '')#remove punctuations
df.des=df.des.replace(r'\s+', ' ', regex=True)#remove multiple spaces

In [ ]:
df.c_tags= df.c_tags.str.replace('|', ' ').str.lower()
df.c_tags = df.c_tags.str.replace(r'[^\w\s]+', '')
df.c_tags = df.c_tags.str.replace('none','')


In [ ]:
df.c_title= df.c_title.str.replace('|', ' ').str.lower()
df.c_title= df.c_title.str.replace('http\S+|www.\S+','',regex=True).str.replace('\r+','',regex= True).str.lower()
df.c_title = df.c_title.str.replace(r'[^\w\s]+', '')

In [ ]:
df.channeltitle= df.channeltitle.str.replace('|', ' ')
#removing punctuation if any from title
df.channeltitle = df.channeltitle.str.replace(r'[^\w\s]+', '')

In [ ]:
pd.set_option('display.max_colwidth', 10)
df.sample(10)

,video_id,trending_date,category_id,publish_time,views,likes,dislikes,comment_count,video_error_or_removed,category_name,des,c_tags,c_title,channeltitle
10187,MSzytv...,18.03.01,24,2017-1...,1546967,17745,789,920,False,Entert...,from t...,snl sa...,cut fo...,Saturd...
10274,KWM4DV...,18.04.01,23,2017-1...,790868,51555,1115,8403,False,Comedy,a lot ...,collee...,my exp...,Psycho...
16797,Z_kfM-...,18.08.02,22,2018-0...,402547,7271,60,695,False,People...,landin...,spacex...,falcon...,Chilly...
30053,HxuRKq...,18.21.04,27,2018-0...,367325,5944,876,1020,False,Education,today ...,fortni...,vinder...,The Ki...
6130,RUbWdN...,17.14.12,25,2017-1...,47746,245,52,330,False,News &...,firefi...,califo...,firefi...,eurone...
28258,LDS8Se...,18.06.04,24,2018-0...,411122,11892,106,482,False,Entert...,heart ...,music ...,westwo...,WaterT...
26071,sfMwXj...,18.26.03,19,2018-0...,434813,7960,243,913,False,Travel...,with s...,cherry...,what s...,INSIDER
37696,r-3iat...,18.29.05,28,2018-0...,6431372,133425,10286,16289,False,Scienc...,fortni...,gaming...,the ul...,Unbox ...
37604,voG07p...,18.29.05,10,2018-0...,10573584,518524,12847,34184,False,Music,get ne...,entert...,zayn ...,ZaynVEVO
1976,6uJf2I...,17.23.11,10,2017-1...,11825470,575900,18051,61498,False,Music,red ve...,red ve...,red ve...,SMTOWN


In [ ]:
def date(d):
    return "20"+d

In [ ]:
df["trending"]=df.trending_date.apply(date)

In [ ]:

df['trending']=pd.to_datetime(df.trending,format="%Y.%d.%m")


In [ ]:
df.drop(columns='trending_date',axis=1,inplace=True)

In [ ]:

df.head(1)

,video_id,category_id,publish_time,views,likes,dislikes,comment_count,video_error_or_removed,category_name,des,c_tags,c_title,channeltitle,trending
0,2kyS6S...,22,2017-1...,748374,57527,2966,15954,False,People...,shante...,shante...,we wan...,CaseyN...,2017-11-14


Preparing for 

In [ ]:

df['text']=df.des + ' ' + df.c_tags + ' ' + df.c_title + ' ' + df.channeltitle

In [ ]:
#Remove common words
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stopwords.words('english'):
            result.append(token)  
    return result

In [ ]:
df['text']=df['text'].apply(preprocess)

In [ ]:
df['text'][0]

['shantells',
 'channel',
 'video',
 'lens',
 'drone',
 'gear',
 'camera',
 'camera',
 'lens',
 'sony',
 'camera',
 'canon',
 'camera',
 'tripod',
 'thing',
 'need',
 'bendy',
 'tripod',
 'lens',
 'expensive',
 'wide',
 'lens',
 'camera',
 'microphone',
 'drone',
 'cheaper',
 'still',
 'great',
 'intro',
 'song',
 'disclosure',
 'ad',
 'selling',
 'promoting',
 'anything',
 'samsung',
 'produce',
 'shantell',
 'video',
 'galaxy',
 'project',
 'initiative',
 'enables',
 'creators',
 'like',
 'shantell',
 'make',
 'projects',
 'might',
 'otherwise',
 'opportunity',
 'make',
 'hope',
 'thats',
 'clear',
 'ask',
 'comments',
 'ill',
 'answer',
 'specifics',
 'shantell',
 'martin',
 'want',
 'talk',
 'marriage',
 'caseyneistat']

In [ ]:
list_of_words = []
for i in df.text:
    for j in i:
        list_of_words.append(j)

In [ ]:
#Get the size of vocabulary 
unique_words=len(list(set(list_of_words)))

In [ ]:
df['text']=df['text'].apply(lambda x: " ".join(x))

In [ ]:
df['text'][1]

'one year presidential election john oliver discusses weve learned far enlists catheter cowboy teach donald trump hasntnnconnect last week tonight last week tonight youtube channel almost news almost happens last week tonight facebook like mom would us twitter news jokes jokes news official site stuff last week tonight trump presidency last week tonight donald trump john oliver trump donald trump trump presidency last week tonight john oliver hbo lastweektonight'

In [ ]:
num_cate=len(df.category_name.unique())

In [ ]:
#Encode the category
from collections import defaultdict
category_to_num=defaultdict()
num_to_category=defaultdict()
category_list=list(df.category_name.unique())
for i,c in enumerate(category_list):
    num_to_category[i]=c
for i,c in enumerate(category_list):
    category_to_num[c]=i

In [ ]:
import tensorflow as tf
import keras 
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM,Bidirectional

In [ ]:
def one_hot_cate(t):
    return tf.keras.utils.to_categorical(category_to_num[t],num_classes=num_cate)

In [ ]:
df['encode_cate']=df.category_name.apply(one_hot_cate)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.text, df.encode_cate, test_size = 0.2)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = unique_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
df['text'].str.len().max()

4468

In [ ]:
from keras.preprocessing.sequence import pad_sequences
padded_train = pad_sequences(train_sequences,maxlen = 40, padding = 'post', truncating = 'post') #Use first 40 words for training; using full size (4468) is the second option.
padded_test = pad_sequences(test_sequences,maxlen = 40, truncating = 'post') 

In [ ]:
model = Sequential()
model.add(Embedding(unique_words, output_dim = 128,input_length = 40))
# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))
# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(16,activation= 'softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
train_y=[]
for i in y_train:
    train_y.append(i)
train_y=np.array(train_y)

In [ ]:
model.fit(padded_train, train_y, batch_size = 128, validation_split = 0.1, epochs = 5)

Epoch 1/5
231/231 [==============================] - 32s 140ms/step - loss: 0.1136 - acc: 0.6361 - val_loss: 0.0450 - val_acc: 0.8944
Epoch 2/5
231/231 [==============================] - 33s 144ms/step - loss: 0.0176 - acc: 0.9613 - val_loss: 0.0136 - val_acc: 0.9747
Epoch 3/5
231/231 [==============================] - 32s 136ms/step - loss: 0.0044 - acc: 0.9916 - val_loss: 0.0101 - val_acc: 0.9820
Epoch 4/5
231/231 [==============================] - 33s 142ms/step - loss: 0.0023 - acc: 0.9954 - val_loss: 0.0089 - val_acc: 0.9863
Epoch 5/5
231/231 [==============================] - 32s 138ms/step - loss: 0.0013 - acc: 0.9976 - val_loss: 0.0081 - val_acc: 0.9890


In [ ]:
pred = model.predict(padded_test)

In [ ]:
pred

array([[4.35303599e-10, 1.41107459e-10, 4.88667817e-09, ...,
        1.40307323e-15, 9.81790062e-13, 4.09713116e-13],
       [8.65601254e-12, 6.58990961e-12, 3.29938230e-11, ...,
        1.08882934e-13, 2.86240476e-12, 5.39925364e-14],
       [1.99761568e-10, 6.31226807e-11, 1.87031968e-09, ...,
        3.37156568e-16, 1.96265543e-13, 1.06735001e-13],
       ...,
       [9.86349002e-10, 3.02453923e-10, 1.78315729e-09, ...,
        9.47933735e-16, 9.58783617e-13, 2.70816735e-13],
       [1.70841008e-10, 2.21931792e-10, 5.02752817e-09, ...,
        6.88099867e-16, 4.08031736e-13, 1.28914078e-13],
       [8.21504115e-11, 1.08550079e-10, 3.79791754e-09, ...,
        5.24926540e-16, 2.48347187e-13, 9.46919138e-14]], dtype=float32)

In [ ]:
#Decode the result
pred_cate=[]
for i in pred:
    pred_cate.append(num_to_category[np.argmax(i)])
    

In [ ]:
from sklearn.metrics import accuracy_score
true_cate=[]
for i in y_test:
    true_cate.append(num_to_category[np.argmax(i)])
accuracy = accuracy_score(true_cate, pred_cate)
print("Model Accuracy : ", accuracy)

Model Accuracy :  0.9764346764346764


In [ ]:
import sklearn.metrics as met
print(met.classification_report(true_cate, pred_cate))

                       precision    recall  f1-score   support

     Autos & Vehicles       1.00      0.95      0.98        85
               Comedy       0.99      0.98      0.98       678
            Education       0.98      0.98      0.98       348
        Entertainment       0.99      0.98      0.98      2021
     Film & Animation       0.95      0.95      0.95       470
               Gaming       1.00      0.97      0.98       173
        Howto & Style       0.99      0.99      0.99       818
                Music       0.99      0.99      0.99      1271
      News & Politics       0.99      0.92      0.96       483
Nonprofits & Activism       0.57      1.00      0.73         4
       People & Blogs       0.93      0.99      0.96       653
       Pets & Animals       0.96      0.93      0.94       174
 Science & Technology       0.90      0.98      0.94       486
                Shows       1.00      1.00      1.00        15
               Sports       0.98      0.98      0.98  